In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
#Kết nối SV_UAT (SQL_SV64)
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri_UAT)

<h3> 1. Lấy bảng

<h5>1.1. Lấy bảng AgentInfo

In [2]:
cutoff = input("Nhập kỳ CUTOFF cuối tháng YYYYMMDD: ")

Nhập kỳ CUTOFF cuối tháng YYYYMMDD: 20230131


In [3]:
agent_query = f"""\n
SELECT * FROM [DPO].[Main_AGENT_INFO_DA_CUTOFF]
WHERE CUTOFF = {cutoff}
"""
agent_table = pd.read_sql(agent_query, conn)
agent_table

,Area_Name,Sales_Unit,Sales_Unit_Code,Parent_Supervisor_Code,Parent_Supervisor_Name,Supervisor_Code,Supervisor_Name,Agent_Number,Agent_Name,Gender,...,Leader,SFC,Leader_count,Appointed_TAPSU,CUTOFF,First_Appointed,Area_Desc,TERRITORY,HOT Name,Sale_Unit_Grade
0,TAA,Ngô Anh Tài,AD082,60030650,DM - PHẠM THỊ CÚC HOA,60030655,FM - PHẠM THỊ HỒNG ANH,60060890,NGUYỄN HOÀNG ANH,Male,...,60030655,,2,2022-06-20,20230131,None,None,None,None,None
1,ABT,Nguyễn Thị Tố Quyên,AD786,60055833,FM - NGUYỄN NGỌC DUY QUANG,60055914,FM - HUỲNH THANH TUẤN,60060917,PHẠM BÁ KHƯƠNG,Male,...,60055914,,3,2022-06-20,20230131,None,None,None,None,None
2,EAK,Thiều Quang Mạnh,AD174,60008686,DM - NGUYỄN VĂN KỶ,60019636,FC - NGUYỄN THỊ NỤ,60021398,NGUYỄN THỊ MẬU,Female,...,60008686,,2,2019-06-19,20230131,None,None,None,None,None
3,AVP,Văn Hương Giang,AD811,,None,60054777,DM - ĐÀO THỊ NGA,60055999,NGUYỄN THỊ BIÊN,Female,...,60054777,,1,2022-01-25,20230131,None,None,None,None,None
4,LAK,Đặng Văn Phan,AD501,60062231,DM - PHAN HỮU SỸ,60075163,FM - ĐẶNG THỊ SOA,60075691,LÊ THỊ KHÁNH LY,Female,...,60075163,,6,2022-12-29,20230131,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,STR,Lục Minh Hiếu,AD409,60002305,FC - LÊ THÀNH DÂN,60000699,AM - TRƯƠNG HUY LUẬN,60006002,NGÔ QUỐC TRUNG,Male,...,None,,2,2016-09-26,20230131,None,None,None,None,None
75902,SG1,Lê Nguyễn Ánh Nguyệt,AD752,60070913,DM - NGUYỄN MINH TUẤN,60070705,FC - LÊ TRƯỜNG KHÁNH,60074423,PHAN THỊ TUYẾT HƯƠNG,Female,...,60070913,,6,2022-12-12,20230131,None,None,None,None,None
75903,BMT,Võ Tất Thắng,AD088,,None,60016183,FC - KIỀU THỊ MỸ LỘC,60016239,LÊ THỊ PHƯỚC,Female,...,None,,1,2018-08-24,20230131,None,None,None,None,None
75904,LAN,Trương Thái Tuấn,AD099,None,None,,None,60005479,HỒ NGỌC THÔNG,Male,...,None,,0,2016-08-25,20230131,None,None,None,None,None


<h5>1.2 Lấy bảng Commision

<h3>2. Xây cấu trúc bảng 

<h5>2.1 Lấy ra quan hệ giữa người giới thiệu và được giới thiệu

In [8]:
introduce_agent = agent_table
introduce_agent = introduce_agent[['Agent Numer of Introducer', 'Agent_Number']]
introduce_agent = introduce_agent.dropna()
introduce_agent = introduce_agent.rename(columns={'Agent Numer of Introducer':"Introducer", 'Agent_Number': "Introducee"})
introduce_agent

,Introducer,Introducee
0,60030655,60060890
1,60055914,60060917
3,60054777,60055999
4,60075163,60075691
5,60035540,60035661
...,...,...
75897,60029911,60031666
75898,60055637,60060274
75899,60055496,60060709
75902,60070705,60074423


<h5>2.2 Mốc lấy thông tin cụ thể của người giới thiệu:

In [16]:
introduce_info = pd.merge(introduce_agent, agent_table, left_on = 'Introducer', right_on ='Agent_Number', how='left')
introduce_info = introduce_info[['Introducer', 'Agent_Name', 'Grade', 'Agent_Status', 'Date_Appointed', 'Introducee']].\
rename(columns={'Agent_Name':'IntroducerName', 'Grade': 'IntroducerGrade', 'Agent_Status':'IntroducerStatus', 'Date_Appointed':'IntroducerAppointed'})
introduce_info

,Introducer,IntroducerName,IntroducerGrade,IntroducerStatus,IntroducerAppointed,Introducee
0,60030655,PHẠM THỊ HỒNG ANH,FM,Enforce,2021-01-11,60060890
1,60055914,HUỲNH THANH TUẤN,FM,Enforce,2022-01-22,60060917
2,60054777,ĐÀO THỊ NGA,DM,Enforce,2022-08-27,60055999
3,60075163,ĐẶNG THỊ SOA,FM,Enforce,2022-12-21,60075691
4,60035540,ĐỖ THỊ HẢI,FC,Enforce,2020-12-25,60035661
...,...,...,...,...,...,...
53357,60029911,NGUYỄN THỊ TRANG,FM,Enforce,2020-09-17,60031666
53358,60055637,LÊ THỊ THANH TUYẾT,FM,Enforce,2022-08-10,60060274
53359,60055496,NGUYỄN HOÀNG TÙNG,FM,Enforce,2022-01-12,60060709
53360,60070705,LÊ TRƯỜNG KHÁNH,FC,Enforce,2022-10-19,60074423
